In [5]:
import os
import pandas as pd


In [7]:
project_dir = '/data/konrad/workspace'

dir_path = project_dir + "/datasets/calfs_observations/feeder"
csv_files = [d for d in os.listdir(dir_path) if d.endswith("csv") and os.path.isfile(os.path.join(dir_path, d))]
# csv_files

In [69]:
columns = ['date', 'start visit', 'end visit', 'animal index', 'animal no.',
       'group', 'station', 'housing weight', 'entitlement act.', 'feed mode',
       'next entitlement at', 'cons. today ml', 'cons. today %',
       'add. 1 today', 'add. 2 today', 'feed speed ml/min', 'visit w. entit.',
       'visit wo. entit.', 'break wo. add.', 'break w. add.', 'total MP g.',
       'total milk', 'failure', 'warnings']
records = []
minutes = 10 * 60
for d in csv_files:
    csv_path = os.path.join(dir_path, d)
    aptms = pd.read_csv(csv_path, delimiter=';', header=0, names=columns).reset_index(drop=True)
    # print(aptms.shape, csv_path)
    # print(aptms.columns)
    aptms["start_time"] = pd.to_timedelta(aptms["start visit"])
    aptms["end_time"] = pd.to_timedelta(aptms["end visit"])
    aptms["timedelta"] = (aptms["end_time"] - aptms["start_time"]).apply(lambda x: x.total_seconds())

    aptms = aptms[aptms["timedelta"] > 0]
    aptms = aptms[aptms["start_time"] < aptms["end_time"]]
    # print(aptms[aptms["timedelta"] <= 0].shape)
    
    # for idx in aptms.sort_values(by="start_time").index.values:
    # print(aptms.index.values)
    for idx, row in aptms.iterrows():
        # key = "{day}_{parc}".format(day=aptms.iloc[idx]["start_time"], parc=aptms.iloc[idx]["station"])
        # row = aptms.iloc[idx]
        # key = "{day}".format(day=row["start_time"])

        after_at =  row["start_time"] + pd.Timedelta(seconds=minutes)
        before_at =  row["start_time"] - pd.Timedelta(seconds=minutes)
        
        records.append({
            "day": d,
            "station": row["station"],
            "start_time": row["start visit"],
            "end_time": row["end visit"],
            "after_at": after_at,
            "before_at": before_at,
            "started_after": aptms[(aptms["start_time"] < after_at) & (aptms["start_time"] > row["start_time"])].shape[0],
            "calf_number_started_after": aptms[(aptms["start_time"] < after_at) & (aptms["start_time"] > row["start_time"])]["animal no."].unique().size,
            # "calf_number_ended_after": aptms[(aptms["end_time"] < after_at) & (aptms["end_time"] > row["start_time"])]["animal no."].unique().size,
            # "ended_after": aptms[(aptms["end_time"] < after_at) & (aptms["end_time"] > row["start_time"])].shape[0],
            "in_after": aptms[(aptms["start_time"] < after_at) & (aptms["end_time"] < after_at) & (aptms["start_time"] > row["start_time"]) & (aptms["end_time"] > row["start_time"])].shape[0],
            "calf_number_in_after": aptms[(aptms["start_time"] < after_at) & (aptms["end_time"] < after_at) & (aptms["start_time"] > row["start_time"]) & (aptms["end_time"] > row["start_time"])]["animal no."].unique().size,
            "started_before": aptms[(aptms["start_time"] > before_at) & (aptms["start_time"] < row["start_time"])].shape[0],
            "calf_number_started_before": aptms[(aptms["start_time"] > before_at) & (aptms["start_time"] < row["start_time"])]["animal no."].unique().size,
            # "ended_before": aptms[(aptms["end_time"] > before_at) & (aptms["end_time"] < row["start_time"])].shape[0],
            # "calf_number_ended_before": aptms[(aptms["end_time"] > before_at) & (aptms["end_time"] < row["start_time"])]["animal no."].unique().size,
            "in_before": aptms[(aptms["start_time"] > before_at) & (aptms["end_time"] > before_at) & (aptms["end_time"] < row["start_time"]) & (aptms["start_time"] < row["start_time"])].shape[0],
            "calf_number_in_before": aptms[(aptms["start_time"] > before_at) & (aptms["end_time"] > before_at) & (aptms["end_time"] < row["start_time"]) & (aptms["start_time"] < row["start_time"])]["animal no."].unique().size,
        })


    # print((aptms["end_time"] - aptms["start_time"]).head())
    # print((aptms["end_time"].total_seconds()).head())
    # print(aptms[(aptms["end_time"] - aptms["start_time"]).total_seconds() > 0],shape)
records = pd.DataFrame(records)
records.to_csv('/data/data_calves/konrad/feeder_aptms_metadata.csv', index=False)
records.head()

,day,station,start_time,end_time,after_at,before_at,started_after,calf_number_started_after,in_after,calf_number_in_after,started_before,calf_number_started_before,in_before,calf_number_in_before
0,F_2022_11_02.csv,1,00:05:23,00:08:24,0 days 00:15:23,-1 days +23:55:23,0,0,0,0,0,0,0,0
1,F_2022_11_02.csv,1,00:30:33,00:35:21,0 days 00:40:33,0 days 00:20:33,5,2,4,2,0,0,0,0
2,F_2022_11_02.csv,1,00:35:21,00:35:36,0 days 00:45:21,0 days 00:25:21,22,3,20,3,1,1,0,0
3,F_2022_11_02.csv,1,00:35:39,00:35:57,0 days 00:45:39,0 days 00:25:39,23,2,21,2,2,1,2,1
4,F_2022_11_02.csv,1,00:37:07,00:38:38,0 days 00:47:07,0 days 00:27:07,24,2,23,2,3,1,3,1


In [70]:
records_sorted = records.sort_values(by=["calf_number_started_after", "started_after"], ascending=False)
records_sorted[["day", "start_time", "started_after", "in_after", "calf_number_started_after", "calf_number_in_after"]].head(20)

,day,start_time,started_after,in_after,calf_number_started_after,calf_number_in_after
8437,F_2022_08_13.csv,07:15:03,13,11,10,8
8439,F_2022_08_13.csv,07:15:33,13,11,10,9
18246,F_2022_09_22.csv,10:39:27,13,12,10,10
18247,F_2022_09_22.csv,10:35:22,12,11,10,9
8046,F_2022_08_14.csv,18:41:28,23,21,9,7
8048,F_2022_08_14.csv,18:41:44,23,21,9,8
69396,F_2022_07_19.csv,08:20:31,23,22,9,8
8051,F_2022_08_14.csv,18:41:50,22,20,9,8
8462,F_2022_08_13.csv,07:35:19,22,20,9,8
8464,F_2022_08_13.csv,07:36:37,22,20,9,8


In [71]:
records_sorted = records.sort_values(by=["started_after", "calf_number_started_after"], ascending=False)
records_sorted[["day", "start_time", "started_after", "in_after", "calf_number_started_after", "calf_number_in_after"]].head(20)

,day,start_time,started_after,in_after,calf_number_started_after,calf_number_in_after
73842,F_2022_08_19.csv,19:17:55,39,37,3,2
73843,F_2022_08_19.csv,19:17:24,38,37,3,2
73844,F_2022_08_19.csv,19:18:11,38,36,3,2
73839,F_2022_08_19.csv,19:13:52,37,35,5,5
73834,F_2022_08_19.csv,19:14:26,37,35,4,4
64614,F_2022_11_19.csv,04:10:55,37,35,3,3
64615,F_2022_11_19.csv,04:12:08,37,36,3,3
64616,F_2022_11_19.csv,04:12:02,37,36,3,3
73838,F_2022_08_19.csv,19:16:30,37,36,3,3
73845,F_2022_08_19.csv,19:18:26,37,35,3,2


In [73]:
drop_idx=[]
visited_day = {}
minutes = 5
records["start_time"] = pd.to_timedelta(records["start_time"])
for idx in records.sort_values(by="start_time").index.values:
    key = "{day}_{parc}".format(day=records.iloc[idx]["day"], parc=records.iloc[idx]["station"])

    if not key in visited_day.keys():
        visited_day[key] = records.iloc[idx]["start_time"]
        continue
    
    diff = (visited_day[key] - records.iloc[idx]["start_time"]).total_seconds()
    
    if diff < (minutes * 60):
        drop_idx.append(idx)
    else:
        visited_day[key] = records.iloc[idx]["start_time"]

spaced_aptm_with_state = records[~records.index.isin(drop_idx)]
spaced_aptm_with_state.reset_index(drop=True, inplace=True)
records_sorted = spaced_aptm_with_state.sort_values(by=["started_after", "calf_number_started_after"], ascending=False)
records_sorted[["day", "start_time", "started_after", "in_after", "calf_number_started_after", "calf_number_in_after"]].head(20)

,day,start_time,started_after,in_after,calf_number_started_after,calf_number_in_after
235,F_2022_10_05.csv,0 days 00:43:40,31,31,5,5
47,F_2022_11_21.csv,0 days 01:54:45,26,25,5,5
1,F_2022_11_02.csv,0 days 00:43:01,23,22,3,2
178,F_2022_12_03.csv,0 days 00:01:40,22,21,8,7
103,F_2022_09_24.csv,0 days 01:26:53,21,21,3,3
179,F_2022_12_03.csv,0 days 00:02:12,20,19,7,6
121,F_2022_09_18.csv,0 days 04:49:43,20,18,5,4
50,F_2022_07_21.csv,0 days 01:56:33,20,19,2,2
140,F_2022_09_20.csv,0 days 00:30:32,20,19,2,2
141,F_2022_09_20.csv,0 days 00:43:56,19,18,3,3


In [74]:
records_sorted = spaced_aptm_with_state.sort_values(by=["calf_number_started_after", "started_after"], ascending=False)
records_sorted[["day", "start_time", "started_after", "in_after", "calf_number_started_after", "calf_number_in_after"]].head(20)

,day,start_time,started_after,in_after,calf_number_started_after,calf_number_in_after
178,F_2022_12_03.csv,0 days 00:01:40,22,21,8,7
179,F_2022_12_03.csv,0 days 00:02:12,20,19,7,6
170,F_2022_09_15.csv,0 days 00:01:29,14,12,6,5
171,F_2022_09_15.csv,0 days 00:03:31,14,12,6,5
235,F_2022_10_05.csv,0 days 00:43:40,31,31,5,5
47,F_2022_11_21.csv,0 days 01:54:45,26,25,5,5
121,F_2022_09_18.csv,0 days 04:49:43,20,18,5,4
63,F_2022_08_30.csv,0 days 00:25:08,17,17,5,5
234,F_2022_10_05.csv,0 days 00:37:15,15,15,5,5
228,F_2022_11_10.csv,0 days 00:51:49,14,12,5,3


In [75]:
spaced_aptm_with_state.to_csv('/data/data_calves/konrad/feeder_aptms_spaced_metadata.csv', index=False)